In [82]:
import tensorflow as tf
import numpy as np

In [83]:
#@title Initialisations
batch_size = 8
teacher_epsilons = [0.1, 0.1]   # len = num_teachers
num_observations = 2

learn_teacher_epsilons = [0, 1]     # bool 
learned_teacher_epsilons = [0.2]    # len = sum(learn_teacher_epsilons)

bounds = 0.8       # threshold on distance to target for teacher regularisation

In [84]:
#@title Create and update the tf teacher epsilons

teacher_epsilons = tf.repeat(tf.convert_to_tensor(
                    [teacher_epsilons]), repeats=batch_size, axis=0)
learned_teacher_epsilons = tf.repeat(tf.convert_to_tensor(
                    [learned_teacher_epsilons]), repeats=batch_size, axis=0)
teacher_epsilons = tf.Variable(teacher_epsilons)
learned_teacher_epsilons = tf.Variable(learned_teacher_epsilons)

# Check for dimension mismatch
tf.debugging.assert_equal(
    tf.shape(learned_teacher_epsilons)[1], sum(learn_teacher_epsilons))

In [85]:
count = 0
for i, x in enumerate(learn_teacher_epsilons):
    print(i)
    if x:
        teacher_epsilons = teacher_epsilons[:, i].assign(
            learned_teacher_epsilons[:, count])
        count += 1

0
1


In [86]:
print(teacher_epsilons)

<tf.Variable 'UnreadVariable' shape=(8, 2) dtype=float32, numpy=
array([[0.1, 0.2],
       [0.1, 0.2],
       [0.1, 0.2],
       [0.1, 0.2],
       [0.1, 0.2],
       [0.1, 0.2],
       [0.1, 0.2],
       [0.1, 0.2]], dtype=float32)>


In [89]:
obs = tf.convert_to_tensor(np.random.randn(batch_size, num_observations))
obs_bool = tf.cast(tf.where(abs(obs) <= bounds, 0, 1), dtype=tf.float32)

# Find if position is in the small square. If so, use both teachers. 
is_insquare = tf.where(obs_bool[:, 0] + obs_bool[:, 1] == 0, 1, 0)
is_insquare = tf.expand_dims(is_insquare, axis=-1)
is_insquare = tf.broadcast_to(is_insquare, [batch_size, num_observations])
is_insquare = tf.cast(is_insquare, dtype = tf.float32)
obs_bool += is_insquare

teacher_epsilons.assign(obs_bool * teacher_epsilons)

<tf.Variable 'UnreadVariable' shape=(8, 2) dtype=float32, numpy=
array([[0. , 0. ],
       [0. , 0.2],
       [0.1, 0. ],
       [0.1, 0. ],
       [0.1, 0.2],
       [0. , 0.2],
       [0.1, 0. ],
       [0.1, 0. ]], dtype=float32)>